### Imports

In [2]:
import requests
from bs4 import BeautifulSoup
import psycopg2

### Connection Test

In [11]:
url = 'https://www.ran.de/fussball/bundesliga/bundesliga-statistiken'
response = requests.get(url)

# Prüfen, ob die Anfrage erfolgreich war (Statuscode 200)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # relevante Daten aus der Webseite extrahieren:
    results = soup.find_all('div', class_='Mannschaft')
    statistics = soup.find_all('div', class_='Passquote')
else:
    print('Fehler beim Abrufen der Webseite')
    
elements = soup.find_all('div',  class_='team-name-list')
element = soup.find('div', id='td')
attribute_value = element['a']
nested_elements = element.find_all('span', class_='nested-class')

TypeError: 'NoneType' object is not subscriptable

### PostgreSQL

In [ ]:
#Verbinden mit der DB
def connect_and_save_data(results, statistics):
    try:
        connection = psycopg2.connect(           
            dbname="PSQL_ADSFS2023Gruppe15",
            user="ADSFS2023Gruppe15",
            password="ADS_FS_2023_G15!?"
            host="localhost",
            ports='5432:5432'
        )

        cursor = connection.cursor()

        # Erstellen der Tabelle für die Passquoten
        create_table_query = '''
            CREATE TABLE IF NOT EXISTS bundesliga_passquoten (
                Mannschafts_ID INTEGER,
                Mannschaft TEXT,
                Passquote INTEGER
                Datum DATE
            );
        '''
        cursor.execute(create_table_query)
        connection.commit()

        # Einfügen der extrahierten Daten in die Tabelle
        for result, statistic in zip(results, statistics):
            insert_data_query = '''
                INSERT INTO bundesliga_results (Mannschaft, Passquote, Datm) VALUES (%s, %s, sysdate);
            '''
            cursor.execute(insert_data_query, (Mannschaft.text, Passquote.text))
        
        connection.commit()

    except (Exception, psycopg2.Error) as error:
        print('Fehler beim Verbinden mit der PostgreSQL-Datenbank:', error)
    finally:
        # Schließe die Verbindung
        if connection:
            cursor.close()
            connection.close()

# Führe die Funktion mit den extrahierten Daten aus
connect_and_save_data(results, statistics)